# Importing functions

In [1]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
import re
import joblib
import random 
import numpy as np
import warnings
import tensorflow as tf 
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import make_column_transformer
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from preprocess_days_stats import preprocess_match_days
from preprocess_time_serie import preprocess_teams, create_time_series_features
from preprocess_time_series_features import preprocess_features_time_series, create_fast_preprocessing_ts, preprocess_features_time_series_odds, create_fast_preprocessing_ts_odds
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [ ]:
def preprocess_features_time_series_odds_preds(df_Serie_A, num_features, percentual_preds=0.95):

    all_features = ['ft_goals','ft_goals_conceded','shots','shots_target', 'fouls_done','corners_obtained', 'yellows', 'reds']
    less_features = ['ft_goals','ft_goals_conceded','shots', 'fouls_done','corners_obtained', 'reds']
    few_features = ['ft_goals','ft_goals_conceded','shots', 'reds']

    Train_df = df_Serie_A.iloc[:10]
    Valid_df = df_Serie_A.iloc[:10]
    Test_df = df_Serie_A.iloc[int(len(df_Serie_A)*percentual_preds):]

    Train_labels = Train_df[['ft_result']]
    Valid_labels = Valid_df[['ft_result']]
    Test_labels = Test_df[['ft_result']]
    
    Train_odds = Train_df[['home_win_odds','draw_odds','away_win_odds']]
    Valid_odds = Valid_df[['home_win_odds','draw_odds','away_win_odds']]
    Test_odds = Test_df[['home_win_odds','draw_odds','away_win_odds']]

    # preprocess Train dataframe
    Train_teams = Train_df[['stagione','hometeam','awayteam']]

    if num_features == 'all':
        features = all_features
        print('utilizzando tutte le features')
    elif num_features == 'less':
        print('utilizzando meno features')
        features = less_features
    else:
        print('utilizzando poche features')
        features=few_features

    Train_dict_features={}

    for feature in features:
        Train_dict_features[feature] = pd.DataFrame({})
        for colonna in Train_df.columns:
            pattern = re.compile(rf'^home_{feature}_\d+$')
            if pattern.match(colonna):
                Train_dict_features[feature][colonna]=Train_df[colonna]
        for colonna in Train_df.columns:
            pattern = re.compile(rf'^away_{feature}_\d+$')
            if pattern.match(colonna):
                Train_dict_features[feature][colonna]=Train_df[colonna]

    #preprocess valid dataframe
    Valid_teams = Valid_df[['stagione','hometeam','awayteam']]


    if num_features == 'all':
        features = all_features
        print('utilizzando tutte le features')
    elif num_features == 'less':
        print('utilizzando meno features')
        features = less_features
    else:
        print('utilizzando poche features')
        features=few_features

    Valid_dict_features={}

    for feature in features:
        Valid_dict_features[feature] = pd.DataFrame({})
        for colonna in Valid_df.columns:
            pattern = re.compile(rf'^home_{feature}_\d+$')
            if pattern.match(colonna):
                Valid_dict_features[feature][colonna]=Valid_df[colonna]
        for colonna in Valid_df.columns:
            pattern = re.compile(rf'^away_{feature}_\d+$')
            if pattern.match(colonna):
                Valid_dict_features[feature][colonna]=Valid_df[colonna]

    # preprocess test dataframe
    Test_teams = Test_df[['stagione','hometeam','awayteam']]

    if num_features == 'all':
        features = all_features
        print('utilizzando tutte le features')
    elif num_features == 'less':
        print('utilizzando meno features')
        features = less_features
    else:
        print('utilizzando poche features')
        features=few_features

    Test_dict_features={}

    for feature in features:
        Test_dict_features[feature] = pd.DataFrame({})
        for colonna in Test_df.columns:
            pattern = re.compile(rf'^home_{feature}_\d+$')
            if pattern.match(colonna):
                Test_dict_features[feature][colonna]=Test_df[colonna]
        for colonna in Test_df.columns:
            pattern = re.compile(rf'^away_{feature}_\d+$')
            if pattern.match(colonna):
                Test_dict_features[feature][colonna]=Test_df[colonna]

    #encoding teams
    # load the  transformer
    teams_transf = joblib.load('transformers/teams_transformer.pkl')

    Train_teams_encoded = teams_transf.transform(Train_teams)
    Valid_teams_encoded = teams_transf.transform(Valid_teams)
    Test_teams_encoded = teams_transf.transform(Test_teams)

    #encoding labels
    # load the  transformer
    ordinal_encoder = joblib.load('transformers/ordinal_encoder_transformer.pkl')

    Train_labels_encoded = np.squeeze(ordinal_encoder.transform(np.array(Train_labels).reshape(-1, 1)))
    Valid_labels_encoded = np.squeeze(ordinal_encoder.transform(np.array(Valid_labels).reshape(-1, 1)))
    Test_labels_encoded = np.squeeze(ordinal_encoder.transform(np.array(Test_labels).reshape(-1, 1)))  

    #encoding numerical features
    Train_dict_features_norm = Train_dict_features.copy()
    Valid_dict_features_norm = Valid_dict_features.copy()
    Test_dict_features_norm = Test_dict_features.copy()

    for feature in list(Train_dict_features.keys()):
        feature_list = Train_dict_features_norm[feature].columns
        numerical_transf = make_column_transformer(
            (MinMaxScaler(), feature_list), 
            sparse_threshold=0  
        )
        numerical_transf.fit(Train_dict_features[feature])

        # save the  transformer
        if save_tranformer: 
         joblib.dump(numerical_transf, 'transformers/numerical_transformer.pkl')

        Train_dict_features_norm[feature] = numerical_transf.transform(Train_dict_features_norm[feature])
        Valid_dict_features_norm[feature] = numerical_transf.transform(Valid_dict_features_norm[feature])
        Test_dict_features_norm[feature] = numerical_transf.transform(Test_dict_features_norm[feature])
    
    # Encoding odds
    odds_transf = make_column_transformer(
        (MinMaxScaler(), ['home_win_odds','draw_odds','away_win_odds']), 
        sparse_threshold=0  
    )

    odds_transf.fit(Train_odds)

    # save the  transformer
    if save_tranformer: 
         joblib.dump(odds_transf, 'transformers/odds_transformer.pkl')

    Train_odds_norm = odds_transf.transform(Train_odds)
    Valid_odds_norm = odds_transf.transform(Valid_odds)
    Test_odds_norm = odds_transf.transform(Test_odds)

    return (Train_teams_encoded, Valid_teams_encoded, Test_teams_encoded, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
            Train_dict_features_norm, Valid_dict_features_norm, Test_dict_features_norm, Train_teams, Valid_teams, Test_teams, Train_labels, Valid_labels, Test_labels, 
            Train_dict_features, Valid_dict_features, Test_dict_features, Train_df, Valid_df, Test_df, 
            Train_odds_norm, Valid_odds_norm, Test_odds_norm)

# Adding the new results to the dataframe

In [2]:
# Leggi il file CSV
df = pd.read_csv(r"C:\Users\Hp\Documents\Serie_A_dump\csv_predictions\december_four.csv", parse_dates=['Date'], dayfirst=True)

# Crea un DataFrame con 10 righe di zeri
new_rows = pd.DataFrame(0, index=range(10), columns=df.columns)

today_date = '2023-12-10'

# Assegna valori specifici alle colonne 'Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam' per ciascuna riga
new_rows.loc[0, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Juventus', 'Napoli']
new_rows.loc[1, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Verona', 'Lazio']
new_rows.loc[2, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Atalanta', 'Milan']
new_rows.loc[3, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Inter', 'Udinese']
new_rows.loc[4, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Frosinone', 'Torino']
new_rows.loc[5, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Monza', 'Genoa']
new_rows.loc[6, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Salernitana', 'Bologna']
new_rows.loc[7, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Roma', 'Fiorentina']
new_rows.loc[8, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Empoli', 'Lecce']
new_rows.loc[9, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Cagliari', 'Sassuolo']


# Aggiungi le nuove righe al DataFrame esistente
new_csv = pd.concat([df, new_rows], ignore_index=True)
new_csv['Date'] = pd.to_datetime(new_csv['Date'], format='%Y-%m-%d')

# Salva il DataFrame aggiornato su un nuovo file CSV
new_csv.to_csv(r"C:\Users\Hp\Serie_A\csv_predictions\december_four.csv")

len(new_csv['HomeTeam'].unique()), len(new_csv['AwayTeam'].unique())


(20, 20)

In [3]:
#preprocess the data
df_giornate = preprocess_match_days(r"C:\Users\Hp\Serie_A\csv_predictions")
num_features = 'less'
num_giornate = 4
Statistiche_squadre_dict = preprocess_teams(dataframe = df_giornate)
df_Serie_A = create_time_series_features(num_features, Statistiche_squadre_dict, df_giornate, num_giornate).dropna()

(Train_teams_encoded, Valid_teams_encoded, Test_teams_encoded, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
    Train_dict_features_norm, Valid_dict_features_norm, Test_dict_features_norm, Train_teams, Valid_teams, Test_teams, Train_labels, Valid_labels, 
    Test_labels, Train_dict_features, Valid_dict_features, Test_dict_features, Train_df, Valid_df, Test_df, 
    Train_odds_norm, Valid_odds_norm, Test_odds_norm) = preprocess_features_time_series_odds(df_Serie_A, num_features, random_state=False)

feature_input_shape = Test_dict_features_norm[list(Test_dict_features_norm.keys())[0]].shape[1]
Train_teams_shape = Test_teams_encoded.shape[1]

Dataset_train_norm, Dataset_valid_norm, Dataset_test_norm = create_fast_preprocessing_ts_odds(Train_teams_encoded, Train_dict_features_norm, Train_labels_encoded,
                                                                                         Valid_teams_encoded, Valid_dict_features_norm,
                                                                    Valid_labels_encoded,Test_teams_encoded, Test_dict_features_norm, Test_labels_encoded, 
                                                                    Train_odds_norm, Valid_odds_norm, Test_odds_norm)

Reading file: december_four.csv
Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!
utilizzando meno features
preprocess finished
utilizzando meno features
utilizzando meno features
utilizzando meno features


In [4]:
#import the model 
odds_model = tf.keras.models.load_model(r'c:\Users\Hp\Serie_A\model_experiments\model_odds_time_series')


In [6]:
Test_labels.shape

(312, 1)

In [8]:
## Visualizziamo un po' di risultati 
model_odds_new_pred_probs = odds_model.predict((Dataset_test_norm))
model_odds_new_prob = model_odds_new_pred_probs.max(axis=1)
model_odds_new_predictions = model_odds_new_pred_probs.argmax(axis=1)
model_odds_new_compare = pd.DataFrame({
                                'stagione': list( Test_df['stagione'] ),
                                'hometeam': list( Test_df['hometeam'] ),
                                'awayteam': list( Test_df['awayteam'] ),
                                'preds': model_odds_new_predictions, 
                                'best_pred_prob': model_odds_new_prob,
                                'home_win_odds': list( Test_df['home_win_odds'] ),
                                'draw_odds': list( Test_df['draw_odds'] ),
                                'away_win_odds': list( Test_df['away_win_odds'])
                                })

ValueError: in user code:

    File "C:\Users\Hp\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Hp\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Hp\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Hp\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\Hp\AppData\Roaming\Python\Python311\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Hp\AppData\Roaming\Python\Python311\site-packages\keras\layers\reshap